# Passo a Passo - Básico

Ler arquivo

In [ ]:
import pandas as pd

file_path = "..."
data = pd.read_csv(file_path)

Escolher o "alvo" (prediction target)

In [ ]:
y = data.nome_coluna_alvo

Escolher as "features", as características

In [ ]:
data_features = ['coluna_x1', 'coluna_x2', 'coluna_x3']

X = data[data_features]

Dividir os dados para treino e validação

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y  = train_test_split(random_state = 0)

Verificar se existem valores faltantes

Existem 3 maiores abordagens:
1) Excluir linhas com dados faltantes
2) Imputação - atribuir aos valores faltantes a média dos valores de cada coluna
3) Extensão da imputação - faz a imputação padrão e também uma nova coluna para mostrar onde a imputação foi feita


Para saber qual a melhor abordagem, podemos testar uma a uma com uma função:


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

def  get_score (train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(random_state = 1)
    model.fit(train_X, train_y)
    preds = model.predict(val_X)
    return mean_absolute_error(val_y, preds)

Primeira abordagem - simplesmente excluir

```python
dados_nulos = []

for coluna in train_X.columns:
    if train_X[coluna].isnull().any():
        # se houver valores NaN, adiciona a lista 
        dados_nulos.append(coluna)

# removendo essas colunas
train_X_reduzido = train_X.drop(dados_nulos, axis=1)
val_X_reduzido = val_X.drop(dados_nulos, axis=1)

print("MAE: ", get_score(train_X_reduzido, val_X_reduzido, train_y, val_y))
```

Segunda abordagem - imputação

```python
from sklearn.impute import SimpleImputer

# imputação
my_imputer = SimpleImputer()
train_X_imput = pd.DataFrame(my_imputer.fit_transform(train_X))
val_X_imput = pd.DataFrame(my_imputer.transform(val_X))

# imputação removeu o nome das colunas
## colocando de volta
train_X_imput.columns = train_X.columns
val_X_imput.columns = val_X.columns

print("MAE: ", get_score(train_X_imput, val_X_imput, train_y, val_y))
```

Terceira abordagem - extensão da imputação

```python
# copiando os dados originais
train_X_plus = train_X.copy()
val_X_plus = val_X.copy()

# fazendo uma nova coluna para os valores NaN
for coluna in dados_nulos:
    train_X_plus[coluna + '_was_missing'] = train_X_plus[coluna].isnull()
    val_X_plus[coluna + '_was_missing'] = val_X_plus[coluna].isnull()

# imputação
my_imputer = SimpleImputer()
imput_X_train_plus = pd.DataFrame(my_imputer.fit_transform(train_X_plus))
imput_X_valid_plus = pd.DataFrame(my_imputer.transform(val_X_plus))

# imputação removeu os nomes das colunas
## renomeando elas...
imput_X_train_plus.columns = train_X_plus.columns
imput_X_valid_plus.columns = val_X_plus.columns

print("MAE: ", get_score(imput_train_X, imput_val_X, train_y, val_y))
```

Tendo em emnte qual abordagem é mais favorável ao estudo, optamos por ela e realizamos o respectivo algoritmo

1) Construir o modelo

In [ ]:
from sklearn.tree import DecisionTreeRegressor
data_model = DecisionTreeRegressor(random_state = 1)

2) Ajustar modelo as variáveis 

In [ ]:

data_model.fit(train_X, train_y)

 3) Fazer previsões


In [ ]:
val_predictions = data_model.predict(val_X)

4) Validar o modelo

Pode ser validado pelo erro médio absoluto (Mean Absolute Error - MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(val_y, val_predictions))

Pré processar os dados de teste:

1) Comparar os resultados obtidos com os resultados atuais/reais

In [ ]:
print('Previsão: ', val_predictions[:5])
print('Atual: ', val_y.head().tolist())

In [ ]:
# # imputação

# final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# # previsões
# preds_test = data_model.predict(final_X_test)

# step_4.b.check()

2) Salvar as previsões em um arquivo


In [ ]:
output = pd.DataFrame({'Id': train_X.index, 'SalePrice': preds})
output.to_csv('submissao.csv', index=False)